In [34]:
import numpy as np 
import torch
import pdb
# from utils.similarity_metrics import local_similarity_map , local_correlation
import matplotlib.pyplot as plt

import seaborn as sns 

In [35]:
noisy_image = np.load('./data/mp36-08_pstm_stk/noisy.npy')[:-1,:-1].astype('float64')
denoised_image = np.load('./data/mp36-08_pstm_stk/denoised_image.npy').astype('float64')
window_size = 9
n_dims = 1
lamb = 0.5 

In [36]:
noisy_image.shape, denoised_image.shape

((3000, 2648), (1, 1, 3000, 2648))

In [37]:
noisy_image.max(), denoised_image.max()

(224395637.6661591, 7073.75244140625)

In [38]:
denoised_image = np.squeeze(denoised_image)
noise = noisy_image - denoised_image

In [39]:
denoised_image.max(), noise.max()

(7073.75244140625, 224388563.9137177)

In [7]:
pad = window_size // 2
pad_width = [[pad, window_size - (1 + pad)], [pad * (n_dims - 1), (window_size - (1 + pad)) * (n_dims - 1)]]
pad, pad_width

(4, [[4, 4], [0, 0]])

In [8]:
image = np.pad(denoised_image, pad_width=pad_width, mode='mean')
image_noise = np.pad(noise, pad_width=pad_width, mode='mean')

window_shape=[window_size, window_size if n_dims == 2 else 1]
image_view = np.lib.stride_tricks.sliding_window_view(image, window_shape=window_shape).astype('float64')
image_noise_view = np.lib.stride_tricks.sliding_window_view(image_noise, window_shape=window_shape,).astype('float64')

straighten = (np.dot(*image_view.shape[:2]), np.dot(*image_view.shape[2:]))
image_view = image_view.reshape(straighten)
image_noise_view = image_noise_view.reshape(straighten)

H = np.eye(window_size**n_dims, dtype='float64') * lamb
print(np.isnan(H).sum())
H = np.lib.stride_tricks.as_strided(H, shape=(image_view.shape[0], window_size**n_dims, window_size**n_dims),
                                        strides=(0, 8 * window_size**n_dims, 8))
print(np.isnan(H).sum())

0
0


In [9]:
image_view.shape[0], window_size**n_dims, window_size**n_dims

(7944000, 9, 9)

In [10]:
0, 8 * window_size**n_dims, 8

(0, 72, 8)

In [11]:
image_view.shape

(7944000, 9)

In [12]:
device = 'cuda'

H = torch.from_numpy(H).to(device=device)

a = torch.from_numpy(image_view).to(device=device)
b = torch.from_numpy(image_noise_view).to(device=device)

In [13]:
A = torch.zeros((*a.shape, a.shape[-1]), dtype=torch.float64).to(device)
B = torch.zeros((*b.shape, b.shape[-1]), dtype=torch.float64).to(device)

A[:] = torch.diag_embed(torch.diagonal(a[:]))
B[:] = torch.diag_embed(torch.diagonal(b[:]))

In [14]:
a.shape,a.dtype, A.shape,A.dtype, b.shape, b.dtype, B.shape, B.dtype

(torch.Size([7944000, 9]),
 torch.float64,
 torch.Size([7944000, 9, 9]),
 torch.float64,
 torch.Size([7944000, 9]),
 torch.float64,
 torch.Size([7944000, 9, 9]),
 torch.float64)

In [15]:
torch.diagonal(a).max()

tensor(7073.7524, device='cuda:0', dtype=torch.float64)

In [16]:
def shaping_conjugate_gradient(L, H, d, lamb, tol=1e-5, N=4):
    """Vectorised Shaping Conjugate gradient Algorithm for a system with smoothing operator.
    Fomel, Sergey. "`Shaping regularization in geophysical-estimation problems
    <https://library.seg.org/doi/10.1190/1.2433716>`_"
    """
#     pdb.set_trace()
    device='cuda'
    p = torch.zeros_like(d).to(device)
    m = torch.zeros_like(d).to(device)
    r = -1 * torch.ones_like(d) * d
    sp = torch.zeros_like(d).to(device)
    sm = torch.zeros_like(d).to(device)
    sr = torch.zeros_like(d).to(device)
    EPS = 1e-5
    
    for i in range(N):
        print(f'Iteration {i}')
        gm = (torch.matmul(torch.permute(L,[0, 2, 1]),r.view(*r.shape, 1))).squeeze() - lamb * m
        gp = (torch.matmul(torch.permute(H,[0, 2, 1]),gm.view(*gm.shape, 1))).squeeze() + lamb * p
        gm = torch.matmul(H , gp.view(*gp.shape, 1))
        gr = torch.matmul(L,gm)

        rho = torch.sum(gp ** 2, axis=1)
        if i == 0:
            beta = torch.zeros((L.shape[0], 1)).to(device)
            rho0 = rho
        else:
            beta = (rho / (rho_hat + EPS)).view(*rho.shape,1)
            if torch.all(beta < tol) or torch.all(rho / (rho0 + EPS) < tol):
                return m

        sp = gp + beta * sp
        sm = gm.squeeze() + beta * sm
        sr = gr.squeeze() + beta * sr

        alpha = rho / (torch.sum(sr ** 2, axis=1) + lamb * torch.sum(sp ** 2, axis=1) - torch.sum(sm ** 2, axis=1) + EPS)
        alpha = alpha.view(*alpha.shape, 1)

        p -= alpha * sp
        m -= alpha * sm
        r -= alpha * sr
        rho_hat = rho
#         pdb.set_trace()
    return m

In [17]:
lamb1 = 1
lamb2 = 1

c1 = shaping_conjugate_gradient(A, H, b, lamb1)
c1 = c1.to(device='cpu')
c2 = shaping_conjugate_gradient(B, H, a, lamb2)
c2 = c2.to(device='cpu')

similarity_map = torch.sum(c1 * c2, axis=1)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 0
Iteration 1
Iteration 2
Iteration 3


In [18]:
c1

tensor([[-8.3630e-01,  1.1602e+00,  5.0419e-01,  ..., -1.5055e+02,
         -4.8595e+01,  7.0353e+02],
        [-1.0341e+00,  1.4344e+00,  6.2331e-01,  ...,  1.2321e+06,
         -1.0274e+02,  4.1564e+03],
        [ 2.6167e+00, -3.6295e+00, -1.5772e+00,  ...,  1.4124e+06,
         -1.4806e+03,  2.2385e+03],
        ...,
        [ 1.0152e+00, -3.1409e+00, -3.8176e+00,  ..., -2.9495e+03,
          6.9566e+00,  6.9566e+00],
        [ 2.7627e+00, -4.4635e+00, -1.8016e+00,  ..., -2.9099e+03,
          7.3096e+00,  7.3096e+00],
        [ 6.3943e-01, -7.1877e+00, -2.6078e+00,  ..., -1.7117e+03,
          5.8328e+00,  5.8328e+00]], dtype=torch.float64)

In [19]:
c2

tensor([[ 5.3604e-01,  6.5012e-01, -1.1491e+00,  ...,  1.7109e-03,
         -4.2279e-05, -1.0815e+00],
        [ 5.3405e-01,  6.4770e-01, -1.1448e+00,  ...,  1.7109e-03,
         -4.2279e-05, -1.0816e+00],
        [ 5.3457e-01,  6.4833e-01, -1.1459e+00,  ...,  1.7109e-03,
         -4.2279e-05, -1.0816e+00],
        ...,
        [ 1.0950e+00,  1.3264e+00, -2.2811e+00,  ..., -3.8252e-01,
          9.2835e-03, -4.2474e-01],
        [ 1.1245e+00,  1.3619e+00, -2.3304e+00,  ..., -3.9165e-01,
          9.2538e-03, -4.1124e-01],
        [ 1.1246e+00,  1.3620e+00, -2.3302e+00,  ..., -3.9055e-01,
          9.2269e-03, -4.0928e-01]], dtype=torch.float64)

In [20]:
sim = torch.sum(c1 * c2, axis=1)

In [21]:
sim = sim.numpy()
sim = sim.reshape((3000,-1))

In [1]:
plt.figure(figsize=(60,20))
image = sns.heatmap(sim / sim.max(), cmap = plt.cm.jet)

NameError: name 'plt' is not defined

In [1]:
import numpy as np
from utils.similarity_metrics import local_similarity_map_gpu , local_correlation

noisy_image = np.load('./data/mp36-08_pstm_stk/noisy.npy')[:-1,:-1].astype('float64')
denoised_image = np.load('./data/mp36-08_pstm_stk/denoised_image.npy').astype('float64')

denoised_image = np.squeeze(denoised_image)
noise = noisy_image - denoised_image

sim_map = local_similarity_map_gpu(denoised_image, noise, lamb=0.5, window_size=9, n_dims=1)

Iteration 0
Iteration 1
Iteration 2
Iteration 3
Iteration 0
Iteration 1
Iteration 2
Iteration 3


In [3]:
sim_map.shape

torch.Size([3000, 2648])